El objetivo de este código es implementar el algoritmo MLP, separando los datos por sujeto.
Va a buscar los datos en una carpeta que se encuentre en el mismo directorio que se llame *datos_baseline*.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [ ]:
# instalar comet ml en caso que sea necesario
!pip install comet_ml

In [ ]:
# Importo las librerías
from lightgbm import LGBMClassifier
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import time
from comet_ml import Experiment
import joblib
import matplotlib.pyplot as plt
import sys
sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [ ]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto los datos
features = load("FEATURES_W200_I50.joblib")
label = load("label_W200_I50.joblib")

features_array = np.array(features)
label_array = np.array(label)

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [ ]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto particiones
folds_ind_lista = []
sujetos_array_lista = []

for i in range(0,10):
  folds_ind_lista.append(joblib.load('folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl'))
  sujetos_array_lista.append(joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl'))

# Aclaracion: sujetos_array va de 0 a 9. Por lo que el sujeto 0 en realidad es el 1 y asi sucesivamente

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [ ]:
for j in range(0,10):

  # Particion de datos de esta iteracion
  folds_ind = folds_ind_lista[j]
  sujetos_array = sujetos_array_lista[j]

  # Voy a usar como sujeto (o fold, es lo mismo) de test el que aparece primero. Mirar primer elemento de sujetos_array, y tener presente
  # la aclaracion hecha en el bloque de codigo anterior

  # por el criterio elegido, se que el primer fold (o sujeto) es para test, asi que lo saco
  folds_ind_test = folds_ind[0]

  # Uno los datos de los folds de entrenamiento
  # me quedo con los folds (o sujetos) de train quitando el primero (posicion 0)
  folds_ind_train = folds_ind[1:]
  # concateno los arrays (folds) en uno solo
  folds_ind_train_array = np.concatenate(folds_ind_train)
  # aplano el array resultante
  folds_ind_train_array = folds_ind_train_array.ravel()

  # Si tuviera mas de un fold de test tendria que hacer lo mismo que hice recien para train, pero no lo hago porque solo tengo uno

  # Defino X_train, y_train, X_test, y_test
  X_train = features_array[folds_ind_train_array, :]
  y_train = label_array[folds_ind_train_array, 1]
  X_test = features_array[folds_ind_test, :]
  y_test = label_array[folds_ind_test, 1]

  start_time = time.time()    # comienzo a medir el tiempo
  # Parametros de entrada de la red neuronal que tenemos que definir
  # Definimos a mano la cantidad de features
  featureNum = 15
  chNum = 16
  # Numero de columnas que va a haber en features
  inputDim = chNum*featureNum
  # Numero de gestos contando el reposo
  clasNum = np.max(label_array[:, 1]) + 1

  # Architecture 250 - 500 - 250
  clf = cl.Clasificador(model= "mlp", arch = [250,500,250], catNum = clasNum, featureNum = inputDim)
  clf.train(X_train, y_train)

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo

  nombre = 'baseline_mlp_sep_sub' + str(j+1) + '_test'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'mlp', 'sep_sub', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("partition_array", sujetos_array)   # Guarda el arreglo en el experimento
  exp.log_text("El primer sujeto de partition_array es el de test, el resto son todos de train. Se usó la particion que se guarda en datos_baseline")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

Epoch 1/50
3686/3686 [==============================] - 16s 4ms/step - loss: 1.5988 - accuracy: 0.4426
Epoch 2/50
3686/3686 [==============================] - 23s 6ms/step - loss: 1.2877 - accuracy: 0.5537
Epoch 3/50
3686/3686 [==============================] - 14s 4ms/step - loss: 1.1316 - accuracy: 0.6073
Epoch 4/50
3686/3686 [==============================] - 14s 4ms/step - loss: 1.0243 - accuracy: 0.6450
Epoch 5/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.9384 - accuracy: 0.6749
Epoch 6/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.8626 - accuracy: 0.7013
Epoch 7/50
3686/3686 [==============================] - 14s 4ms/step - loss: 0.8111 - accuracy: 0.7169
Epoch 8/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.7609 - accuracy: 0.7336
Epoch 9/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.7114 - accuracy: 0.7504
Epoch 10/50
3686/3686 [==============================] - 14s 4ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/46afbb461f64444fb1c6dc33edc95863

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub1_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/46afbb461f64444fb1c6dc33edc95863
COMET INFO

Epoch 1/50
3623/3623 [==============================] - 15s 4ms/step - loss: 1.5858 - accuracy: 0.4506
Epoch 2/50
3623/3623 [==============================] - 15s 4ms/step - loss: 1.2824 - accuracy: 0.5578
Epoch 3/50
3623/3623 [==============================] - 15s 4ms/step - loss: 1.1213 - accuracy: 0.6142
Epoch 4/50
3623/3623 [==============================] - 15s 4ms/step - loss: 1.0092 - accuracy: 0.6513
Epoch 5/50
3623/3623 [==============================] - 15s 4ms/step - loss: 0.9254 - accuracy: 0.6796
Epoch 6/50
3623/3623 [==============================] - 15s 4ms/step - loss: 0.8549 - accuracy: 0.7029
Epoch 7/50
3623/3623 [==============================] - 15s 4ms/step - loss: 0.7962 - accuracy: 0.7209
Epoch 8/50
3623/3623 [==============================] - 18s 5ms/step - loss: 0.7443 - accuracy: 0.7384
Epoch 9/50
3623/3623 [==============================] - 14s 4ms/step - loss: 0.6986 - accuracy: 0.7537
Epoch 10/50
3623/3623 [==============================] - 14s 4ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/e4eb03b4aafc4a189aeb6eeade90546f

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub2_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/e4eb03b4aafc4a189aeb6eeade90546f
COMET INFO

Epoch 1/50
3686/3686 [==============================] - 16s 4ms/step - loss: 1.5891 - accuracy: 0.4409
Epoch 2/50
3686/3686 [==============================] - 17s 5ms/step - loss: 1.2853 - accuracy: 0.5523
Epoch 3/50
3686/3686 [==============================] - 15s 4ms/step - loss: 1.1235 - accuracy: 0.6080
Epoch 4/50
3686/3686 [==============================] - 15s 4ms/step - loss: 1.0065 - accuracy: 0.6499
Epoch 5/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.9175 - accuracy: 0.6794
Epoch 6/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.8459 - accuracy: 0.7036
Epoch 7/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.7897 - accuracy: 0.7226
Epoch 8/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.7401 - accuracy: 0.7379
Epoch 9/50
3686/3686 [==============================] - 15s 4ms/step - loss: 0.6938 - accuracy: 0.7552
Epoch 10/50
3686/3686 [==============================] - 15s 4ms/step - l

...

Epoch 1/50
3626/3626 [==============================] - 16s 4ms/step - loss: 1.5777 - accuracy: 0.4395
Epoch 2/50
3626/3626 [==============================] - 15s 4ms/step - loss: 1.2678 - accuracy: 0.5583
Epoch 3/50
3626/3626 [==============================] - 15s 4ms/step - loss: 1.1148 - accuracy: 0.6126
Epoch 4/50
3626/3626 [==============================] - 15s 4ms/step - loss: 1.0045 - accuracy: 0.6501
Epoch 5/50
3626/3626 [==============================] - 15s 4ms/step - loss: 0.9188 - accuracy: 0.6808
Epoch 6/50
3626/3626 [==============================] - 15s 4ms/step - loss: 0.8489 - accuracy: 0.7036
Epoch 7/50
3626/3626 [==============================] - 15s 4ms/step - loss: 0.7919 - accuracy: 0.7230
Epoch 8/50
3626/3626 [==============================] - 15s 4ms/step - loss: 0.7401 - accuracy: 0.7415
Epoch 9/50
3626/3626 [==============================] - 15s 4ms/step - loss: 0.6972 - accuracy: 0.7547
Epoch 10/50
3626/3626 [==============================] - 15s 4ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/d072a25c245a4c0d8cbc78c7a795e998

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub9_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/d072a25c245a4c0d8cbc78c7a795e998
COMET INFO

Epoch 1/50
3616/3616 [==============================] - 15s 4ms/step - loss: 1.5632 - accuracy: 0.4503
Epoch 2/50
3616/3616 [==============================] - 15s 4ms/step - loss: 1.2580 - accuracy: 0.5638
Epoch 3/50
3616/3616 [==============================] - 15s 4ms/step - loss: 1.1119 - accuracy: 0.6118
Epoch 4/50
3616/3616 [==============================] - 14s 4ms/step - loss: 1.0026 - accuracy: 0.6502
Epoch 5/50
3616/3616 [==============================] - 15s 4ms/step - loss: 0.9174 - accuracy: 0.6796
Epoch 6/50
3616/3616 [==============================] - 15s 4ms/step - loss: 0.8462 - accuracy: 0.7042
Epoch 7/50
3616/3616 [==============================] - 15s 4ms/step - loss: 0.7874 - accuracy: 0.7241
Epoch 8/50
3616/3616 [==============================] - 17s 5ms/step - loss: 0.7393 - accuracy: 0.7396
Epoch 9/50
3616/3616 [==============================] - 15s 4ms/step - loss: 0.6975 - accuracy: 0.7534
Epoch 10/50
3616/3616 [==============================] - 15s 4ms/step - l

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/b14300fad55a4d36bd5dfc569ee291f5

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_sub10_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/b14300fad55a4d36bd5dfc569ee291f5
COMET INF

<Figure size 640x480 with 0 Axes>

In [ ]:
start_time = time.time()    # comienzo a medir el tiempo

# Crear una instancia de LGBMClassifier
clf = LGBMClassifier()

# Entrenar el clasificador
clf.fit(X_train, y_train, eval_metric='logloss')

# Finalizo la medida del tiempo y calculo el tiempo de entrenamiento
end_time = time.time()
training_time = end_time - start_time

# # Realizar predicciones en el conjunto de validación
# y_pred = clf.predict(X_val)

# # Calcular la precisión
# accuracy = accuracy_score(y_val, y_pred)
# print(f'Accuracy: {accuracy}')

# Obtener las iteraciones del mejor modelo
best_iteration = clf.best_iteration_

# Imprimir la advertencia si existe
if hasattr(clf, 'best_iteration_') and clf.best_iteration_ is None:
    print("[LightGBM] [Warning] No further splits with positive gain, best gain: -inf")



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.216863 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50308
[LightGBM] [Info] Number of data points in the train set: 37655, number of used features: 240
[LightGBM] [Info] Start training from score -2.565481
[LightGBM] [Info] Start training from score -2.592793
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.591020
[LightGBM] [Info] Start training from score -2.573805
[LightGBM] [Info] Start training from score -2.543614
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.376846
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.740986
Accurac

Voy a guardar el clasificador, y evaluar distintas métricas: accuracy, precision y recall. Voy a hacer una matriz de confusión.
Por otra parte, voy a guardar la partición de los datos para hacer reproducible el experimento.

In [ ]:
# Guardar el modelo entrenado en un archivo
joblib.dump(bst, 'baseline_gbm_sep_sub_r1.pkl')

# Predecir en el conjunto de test
y_pred = bst.predict(X_test)

# Calcular métricas de desempeño
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar las métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Guardar la imagen de la matriz de confusión
plt.savefig("confusion_matrix.png")

Voy a guardar las métricas calculadas en un experimento en Comet

In [ ]:
# Conectar con Comet
API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO' # Se puede encontrar en Settings(Arriba a la derecha en Comet)

# Crear un experimento con mi API KEY
exp = Experiment(api_key=API_KEY,
                 project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                 auto_param_logging=False)
exp.set_name('baseline_gbm_sep_sub_r1') # Nombre de este experimento
exp.add_tags(['baseline', 'gbm', 'sep_sub']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(y_test, y_pred)
exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
exp.log_text("Primeros dos sujetos --> test, tercero y cuarto --> validación, resto --> train. \n Corresponde a la primera ronda que entreno con estos parámetros. ")   # Comentario del experimento

In [ ]:
# Subir el modelo
exp.log_model(name="baseline_gbm_sep_sub_r1", file_or_folder="baseline_gbm_sep_sub_r1.pkl")
exp.end()

Junto todo en un bloque de código

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"

for k in range(1,2):
  label = np.array(label)
  cantSujetos = np.max(label[:, 2])
  sujeto_test = k
  sorteo = np.random.permutation(cantSujetos) + 1
  sorteo_sin_test = np.delete(sorteo, np.where(sorteo == sujeto_test))
  indices_test = list(np.where(label[:, 2]==sujeto_test)[0])
  indices_val = list(np.where(label[:, 2]==sorteo_sin_test[0])[0])
  indices_val.extend(list(np.where(label[:, 2]==sorteo_sin_test[1])[0]))

  indices_train = []
  for j in sorteo_sin_test[2:]:
      indices_train.extend(np.where(label[:, 2]==j)[0])
  features = np.array(features)

  X_train = features[indices_train, :]
  y_train = label[indices_train, 1]
  X_val = features[indices_val, :]
  y_val = label[indices_val, 1]
  X_test =  features[indices_test, :]
  y_test = label[indices_test, 1]

  start_time = time.time()

  # implemento gbm
  clf = LGBMClassifier()
  clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss')

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo
  bst = clf.best_iteration_

  nombre = 'baseline_gbm_sep_sub' + str(k) + '_testing'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'gbm', 'sep_sub', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
  exp.log_text("Se fija el sujeto de test y se sortea el resto. Los primeros dos sujetos del sorteo son los de validación.")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

/content/drive/My Drive/repo_tesis/archivos_generados_codigos
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.541994 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50345
[LightGBM] [Info] Number of data points in the train set: 45226, number of used features: 240
[LightGBM] [Info] Start training from score -2.565215
[LightGBM] [Info] Start training from score -2.600029
[LightGBM] [Info] Start training from score -2.605103
[LightGBM] [Info] Start training from score -2.578238
[LightGBM] [Info] Start training from score -2.572718
[LightGBM] [Info] Start training from score -2.544725
[LightGBM] [Info] Start training from score -2.544161
[LightGBM] [Info] Start training from score -2.547263
[LightGBM] [Info] Start training from score -2.552927
[LightGBM] [Info] Start training from score -2.595573
[LightGBM] [Info] Start training from score -2.382079
[LightGBM] [Info] Start training from score -2.553779

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/bd0cddec9e424fa4bd23561cf03d600f

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/bd0cddec9e424fa4bd23561cf03d600f
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.25368837711406983
COMET INFO:     precision     : 0.26482242684700047
COMET INFO:  

<Figure size 640x480 with 0 Axes>